In [ ]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [ ]:
%store -r tarjetas
%store -r bancos

In [ ]:
tarjeta_mastercard_debit = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Master Debit'] #Filtro por la Tarjeta Mastercard Debit
tarjeta_mastercard_debit = tarjeta_mastercard_debit.loc[:, (tarjeta_mastercard_debit != 0).any(axis=0)]

In [ ]:
tarjeta_mastercard_debit["RETENCION ING.BRUTOS SIRTAC"] = tarjeta_mastercard_debit[
                        ["RETENCION ING.BRUTOS SIRTAC", "CARGO POSNET WWW.FISERV.COM.AR"]].sum(axis=1, min_count=1) #junto columnas relacionadas en una
tarjeta_mastercard_debit["TOTAL DEDUCCIONES"] = tarjeta_mastercard_debit[
                        ["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS"]].sum(axis=1, min_count=1) #junto columnas relacionadas en una
tarjeta_mastercard_debit = tarjeta_mastercard_debit.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
       'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC','PERCEPCION IVA R.G. 2408   3,00 %', 'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION'])
         #reordeno columnas, aca ya quedan en el orden que es necesario

tarjeta_mastercard_debit.insert(7, 'Iva 21', 
                                tarjeta_mastercard_debit['IVA CRED.FISC.COMERCIO S/ARANC 21,00%']/tarjeta_mastercard_debit['ARANCEL'])
                                #agrego columna de control para IVA %2
tarjeta_mastercard_debit = tarjeta_mastercard_debit.assign(Diferencia = tarjeta_mastercard_debit.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_mastercard_debit.iloc[:, 8:14].sum(axis=True)
                         - tarjeta_mastercard_debit['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_mastercard_debit = tarjeta_mastercard_debit.round(2)


In [ ]:
sum_columns = ['Nro Liquidacion', 'VENTAS C/DESCUENTO CONTADO', 'ARANCEL', 'IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 'Iva 21', 'RETENCION IVA',
               'PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS', 'RETENCION ING.BRUTOS SIRTAC', 
               'PERCEPCION IVA R.G. 2408   3,00 %', 'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_mastercard_debit=tarjeta_mastercard_debit.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [ ]:
tarjeta_mastercard_debit.drop(['Diferencia', 'Iva 21'], inplace=True, axis=1)

#Aca termina. Hay que exportar la tabla a un Excel

In [ ]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\10-2022\Masterdebit1022.xlsx')
tarjeta_mastercard_debit.to_excel(writer, 'Hoja de datos', index=True)
writer.save()